# 🔬 Optimisation Multi-Stratégies : Envelope Parameters

Ce notebook optimise les **paramètres de trading** de la stratégie multi-envelope :
- `ma_base_window` : Réactivité du signal
- `envelopes` : Distance d'entrée
- `size` : Risque par trade  
- `stop_loss` : Protection

**Méthodologie** : Walk-Forward Optimization avec Expanding Window pour éviter l'overfitting.

**⚠️ Important** : L'optimisation de la détection de régime se fait dans `multi_envelope_adaptive.ipynb`.

## 1️⃣ Configuration et chargement des données

In [ ]:
import sys
sys.path.append('../..')

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from itertools import product
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from concurrent.futures import ProcessPoolExecutor, as_completed  # Pour multi-core

# Backtest engine
from utilities.strategies.envelopeMulti_v2 import EnvelopeMulti_v2
from utilities.data_manager import ExchangeDataManager

# Système adaptatif
from core import calculate_regime_series, DEFAULT_PARAMS
from core.params_adapter import FixedParamsAdapter, RegimeBasedAdapter
from core.backtest_comparator import BacktestComparator

# Config plotting
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("✅ Imports réussis (multi-core activé)")

In [ ]:
# ======================
# CONFIGURATION GLOBALE
# ======================

BACKTEST_LEVERAGE = 10
INITIAL_WALLET = 1000
EXCHANGE = "binance"

# Périodes (Expanding Window)
# Couvre tous les cycles: BULL 2020-2021, BEAR 2022, RECOVERY 2023, BULL 2024
PERIODS = {
    "train_full": {"start": "2020-01-01", "end": "2024-06-30"},  # Optimisation
    "holdout": {"start": "2024-07-01", "end": "2024-12-31"},     # Validation finale (intouchable)
}

# Walk-Forward Folds (Expanding Window)
WF_FOLDS = [
    {"train_start": "2020-01-01", "train_end": "2021-12-31", "test_start": "2022-01-01", "test_end": "2022-06-30", "name": "Fold1_Bull2020-21→Bear2022"},
    {"train_start": "2020-01-01", "train_end": "2022-06-30", "test_start": "2022-07-01", "test_end": "2022-12-31", "name": "Fold2_Bull+Bear→Bear"},
    {"train_start": "2020-01-01", "train_end": "2022-12-31", "test_start": "2023-01-01", "test_end": "2023-06-30", "name": "Fold3_Full→Recovery"},
    {"train_start": "2020-01-01", "train_end": "2023-06-30", "test_start": "2023-07-01", "test_end": "2023-12-31", "name": "Fold4_Recovery→Bull2023"},
    {"train_start": "2020-01-01", "train_end": "2023-12-31", "test_start": "2024-01-01", "test_end": "2024-06-30", "name": "Fold5_Full→Bull2024"},
]

# Échantillon stratifié de 8 paires représentatives
# (couvre majors, mid-caps, volatiles, low performers)
PAIRS = [
    "BTC/USDT:USDT",   # Major
    "ETH/USDT:USDT",   # Major
    "SOL/USDT:USDT",   # Mid-cap
    "AVAX/USDT:USDT",  # Mid-cap
    "ADA/USDT:USDT",   # Mid-cap
    "DOGE/USDT:USDT",  # Volatile
    "SUSHI/USDT:USDT", # Volatile (meilleur performer historique)
    "TRX/USDT:USDT",   # Low performer
]

# Classification par profil
PAIR_CLASSES = {
    "BTC/USDT:USDT": "major",
    "ETH/USDT:USDT": "major",
    "SOL/USDT:USDT": "mid-cap",
    "AVAX/USDT:USDT": "mid-cap",
    "ADA/USDT:USDT": "mid-cap",
    "DOGE/USDT:USDT": "volatile",
    "SUSHI/USDT:USDT": "volatile",
    "TRX/USDT:USDT": "low",
}

# Paramètres de backtest communs
BACKTEST_PARAMS = {
    "initial_wallet": INITIAL_WALLET,
    "leverage": BACKTEST_LEVERAGE,
    "maker_fee": 0.0002,
    "taker_fee": 0.0006,
    "reinvest": True,
    "liquidation": True,
    "risk_mode": "scaling",
}

print(f"✅ Configuration chargée")
print(f"   Période: {PERIODS['train_full']['start']} → {PERIODS['holdout']['end']} (BULL 2020-21, BEAR 2022, RECOVERY 2023, BULL 2024)")
print(f"   Paires: {len(PAIRS)} (échantillon stratifié)")
print(f"   - Majors: {sum(1 for c in PAIR_CLASSES.values() if c == 'major')}")
print(f"   - Mid-caps: {sum(1 for c in PAIR_CLASSES.values() if c == 'mid-cap')}")
print(f"   - Volatiles: {sum(1 for c in PAIR_CLASSES.values() if c == 'volatile')}")
print(f"   - Low performers: {sum(1 for c in PAIR_CLASSES.values() if c == 'low')}")
print(f"   Walk-Forward Folds: {len(WF_FOLDS)}")
print(f"   Hold-out: {PERIODS['holdout']['start']} → {PERIODS['holdout']['end']}")

In [ ]:
# Chargement des données
exchange = ExchangeDataManager(
    exchange_name=EXCHANGE,
    path_download="../database/exchanges"  # Pointe vers Backtest-Tools-V2/database/exchanges
)

# Charger TOUTES les données nécessaires (2020-2024 - couvre tous les cycles)
start_date = "2020-01-01"
end_date = "2024-12-31"

df_list_full = {}
print("📥 Chargement des données...")
for pair in tqdm(PAIRS, desc="Paires"):
    df = exchange.load_data(pair, "1h", start_date=start_date, end_date=end_date)
    df_list_full[pair] = df

# BTC pour détection de régime
df_btc_full = exchange.load_data("BTC/USDT:USDT", "1h", start_date=start_date, end_date=end_date)

oldest_pair = min(df_list_full, key=lambda p: df_list_full[p].index.min())

print(f"\n✅ Données chargées")
print(f"   Période: {start_date} → {end_date} (BULL 2020-21, BEAR 2022, RECOVERY 2023, BULL 2024)")
print(f"   Paire la plus ancienne: {oldest_pair}")
print(f"   Nombre de barres: {len(df_list_full[oldest_pair])}")

## 2️⃣ Comparaison manuelle de configurations pré-définies

Teste 5 configurations fixes pour comprendre l'impact des paramètres.

In [ ]:
# Configurations pré-définies
MANUAL_CONFIGS = {
    "Conservative": {
        "ma_base_window": 10,
        "envelopes": [0.05, 0.08, 0.12],
        "size": 0.06,
        "stop_loss": 0.20,
    },
    "Standard (Live actuel)": {
        "ma_base_window": 7,
        "envelopes": [0.07, 0.10, 0.15],
        "size": 0.10,
        "stop_loss": 0.25,
    },
    "Aggressive": {
        "ma_base_window": 5,
        "envelopes": [0.09, 0.13, 0.18],
        "size": 0.12,
        "stop_loss": 0.30,
    },
    "Wide Envelopes": {
        "ma_base_window": 7,
        "envelopes": [0.10, 0.15, 0.20],
        "size": 0.08,
        "stop_loss": 0.25,
    },
    "Tight Envelopes": {
        "ma_base_window": 7,
        "envelopes": [0.05, 0.07, 0.10],
        "size": 0.08,
        "stop_loss": 0.25,
    },
}

print(f"📋 {len(MANUAL_CONFIGS)} configurations manuelles définies")
for name, cfg in MANUAL_CONFIGS.items():
    print(f"   - {name}: MA={cfg['ma_base_window']}, Env={cfg['envelopes']}, Size={cfg['size']}")

In [ ]:
# Fonction helper pour run un backtest
def run_single_backtest(df_list, oldest_pair, params_coin, stop_loss, params_adapter=None):
    """
    Exécute un backtest avec les paramètres donnés.
    
    Returns:
        dict: Résultat du backtest (trades, days, wallet, metrics)
    """
    strategy = EnvelopeMulti_v2(
        df_list=df_list,
        oldest_pair=oldest_pair,
        type=["long", "short"],
        params=params_coin
    )
    
    strategy.populate_indicators()
    strategy.populate_buy_sell()
    
    result = strategy.run_backtest(
        **BACKTEST_PARAMS,
        stop_loss=stop_loss,
        params_adapter=params_adapter
    )
    
    return result

print("✅ Fonction run_single_backtest définie")

In [ ]:
# ======================
# PARALLÉLISATION MULTI-CORE
# ======================

def run_backtest_worker(args):
    """
    Worker function pour exécution parallèle d'un backtest.
    Doit être une fonction top-level pour être pickable par ProcessPoolExecutor.
    
    Args:
        args: tuple (config_dict, df_list, params_coin, stop_loss, adapter_params, is_adaptive)
    
    Returns:
        dict: Résultat du backtest avec métadonnées
    """
    config_dict, df_list_dict, params_coin, stop_loss, adapter_params, is_adaptive = args
    
    # Reconstituer les DataFrames depuis les dicts
    df_list = {pair: pd.DataFrame(data) for pair, data in df_list_dict.items()}
    
    # Trouver oldest_pair
    oldest_pair = min(df_list, key=lambda p: df_list[p].index.min())
    
    # Créer l'adapter
    if is_adaptive:
        regime_series = pd.Series(adapter_params['regime_data'], 
                                  index=pd.DatetimeIndex(adapter_params['regime_index']))
        adapter = RegimeBasedAdapter(
            base_params=params_coin,
            regime_series=regime_series,
            regime_params=DEFAULT_PARAMS,
            multipliers=adapter_params['multipliers'],
            base_std=adapter_params['base_std']
        )
    else:
        adapter = FixedParamsAdapter(params_coin)
    
    # Exécuter backtest
    strategy = EnvelopeMulti_v2(
        df_list=df_list,
        oldest_pair=oldest_pair,
        type=["long", "short"],
        params=params_coin
    )
    
    strategy.populate_indicators()
    strategy.populate_buy_sell()
    
    result = strategy.run_backtest(
        **BACKTEST_PARAMS,
        stop_loss=stop_loss,
        params_adapter=adapter
    )
    
    return {
        'config': config_dict,
        'wallet': result['days']['wallet'].iloc[-1] if len(result['days']) > 0 else INITIAL_WALLET,
        'sharpe': result.get('sharpe_ratio', 0),
        'n_trades': len(result['trades']),
        'result': result  # Garder le résultat complet
    }


def run_backtests_parallel(configs, df_list, regime_series=None, max_workers=None):
    """
    Exécute plusieurs backtests en parallèle sur plusieurs cores CPU.
    
    Args:
        configs: list de dicts avec les paramètres de config
        df_list: dict de DataFrames par paire
        regime_series: Series des régimes (si adaptive)
        max_workers: nombre de workers (None = auto)
    
    Returns:
        list: Résultats des backtests
    """
    # Préparer arguments pour chaque worker
    tasks = []
    
    for config in configs:
        # Convertir DataFrames en dicts pour serialization
        df_list_dict = {pair: df.to_dict('list') for pair, df in df_list.items()}
        
        # Préparer params_coin
        params_coin = {}
        for pair in df_list.keys():
            params_coin[pair] = {
                "src": "close",
                "ma_base_window": config['ma_window'],
                "envelopes": config['envelopes'],
                "size": config['size'] / BACKTEST_LEVERAGE
            }
        
        # Adapter params
        if config['adaptive']:
            adapter_params = {
                'regime_data': regime_series.values.tolist(),
                'regime_index': regime_series.index.tolist(),
                'multipliers': {'envelope_std': True},
                'base_std': 0.10
            }
        else:
            adapter_params = None
        
        tasks.append((
            config,
            df_list_dict,
            params_coin,
            config['stop_loss'],
            adapter_params,
            config['adaptive']
        ))
    
    # Exécution parallèle
    results = []
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(run_backtest_worker, task): i for i, task in enumerate(tasks)}
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="Backtests parallèles"):
            try:
                result = future.result()
                results.append(result)
            except Exception as e:
                print(f"Erreur backtest: {e}")
                results.append(None)
    
    return results


print("✅ Fonctions de parallélisation définies")
print(f"   Gain attendu: ~4-5x avec votre i9-14900HX")

In [ ]:
# Calculer les régimes sur TOUTE la période (pour comparaison manuelle seulement)# ⚠️ Pour Walk-Forward, on recalculera par foldregime_series_full = calculate_regime_series(df_btc_full, confirm_n=12)print("📊 Distribution des régimes (2020-2024):")regime_counts = regime_series_full.value_counts(normalize=True) * 100for regime, pct in regime_counts.items():    print(f"   {regime.name}: {pct:.1f}%")

In [ ]:
# Comparaison manuelle : Fixed vs Adaptive pour chaque config
comparator_manual = BacktestComparator(initial_wallet=INITIAL_WALLET)

print("\n🚀 Exécution des backtests manuels (Fixed + Adaptive)...\n")
print("=" * 80)

for config_name, config in tqdm(MANUAL_CONFIGS.items(), desc="Configurations"):
    # Préparer params_coin
    params_coin = {}
    for pair in PAIRS:
        params_coin[pair] = {
            "src": "close",
            "ma_base_window": config["ma_base_window"],
            "envelopes": config["envelopes"],
            "size": config["size"] / BACKTEST_LEVERAGE
        }
    
    # 1. Fixed params
    adapter_fixed = FixedParamsAdapter(params_coin)
    result_fixed = run_single_backtest(
        df_list_full, oldest_pair, params_coin, 
        config["stop_loss"], adapter_fixed
    )
    
    comparator_manual.add_backtest(
        name=f"{config_name} (Fixed)",
        df_trades=result_fixed['trades'],
        df_days=result_fixed['days'],
        metadata={"config": config, "adaptive": False}
    )
    
    # 2. Adaptive params
    adapter_adaptive = RegimeBasedAdapter(
        base_params=params_coin,
        regime_series=regime_series_full,
        regime_params=DEFAULT_PARAMS,
        multipliers={'envelope_std': True},
        base_std=0.10
    )
    result_adaptive = run_single_backtest(
        df_list_full, oldest_pair, params_coin,
        config["stop_loss"], adapter_adaptive
    )
    
    comparator_manual.add_backtest(
        name=f"{config_name} (Adaptive)",
        df_trades=result_adaptive['trades'],
        df_days=result_adaptive['days'],
        metadata={"config": config, "adaptive": True}
    )

print("\n" + "=" * 80)
print("✅ Backtests manuels terminés\n")

In [ ]:
# Afficher les résultats
comparator_manual.print_summary()

In [ ]:
# Sauvegarder les résultats manuels
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
comparator_manual.save_comparison(f"results_manual_{timestamp}.csv")
print(f"💾 Résultats sauvegardés: results_manual_{timestamp}.csv")

## 3️⃣ Walk-Forward Optimization (Grid Search)

Teste toutes les combinaisons de paramètres avec validation robuste.

In [ ]:
# Grid de paramètres (RÉDUIT pour test rapide)# Une fois validé, tu pourras élargir le gridPARAM_GRID = {    "ma_base_window": [7],  # Valeur live actuelle uniquement    "envelope_sets": [        [0.07, 0.10, 0.15],  # Standard (valeur live)    ],    "size": [0.08, 0.10],  # Teste 2 tailles seulement    "stop_loss": [0.25],  # Valeur live uniquement}# Générer toutes les combinaisonsgrid_combinations = list(product(    PARAM_GRID["ma_base_window"],    PARAM_GRID["envelope_sets"],    PARAM_GRID["size"],    PARAM_GRID["stop_loss"]))print(f"🔍 Grid Search (TEST RAPIDE - Grid réduit)")print(f"   Combinaisons: {len(grid_combinations)}")print(f"   Walk-Forward Folds: {len(WF_FOLDS)}")print(f"   Total backtests: {len(grid_combinations) * len(WF_FOLDS) * 2} (fixed + adaptive)")print(f"   Temps estimé: ~{len(grid_combinations) * len(WF_FOLDS) * 2 * 3 / 60:.0f} min ")\nprint(f"   Période: 2020-2024 (couvre BULL 2020-21, BEAR 2022, RECOVERY 2023, BULL 2024)")print(f"\n💡 Pour explorer plus de configs, élargir le grid après validation")

In [ ]:
# Fonction pour filtrer DataFrame par dates
def filter_df_by_dates(df, start_date, end_date):
    """Filtre un DataFrame par dates."""
    mask = (df.index >= pd.Timestamp(start_date)) & (df.index <= pd.Timestamp(end_date))
    return df[mask]

def filter_df_list_by_dates(df_list, start_date, end_date):
    """Filtre un dict de DataFrames par dates."""
    return {pair: filter_df_by_dates(df, start_date, end_date) for pair, df in df_list.items()}

print("✅ Fonctions de filtrage définies")

In [ ]:
# Fonction de calcul du score composite anti-overfitting
def calculate_composite_score(bt_result, train_sharpe=None):
    """
    Calcule le score composite pour évaluer une configuration.
    
    Args:
        bt_result: Résultat du backtest
        train_sharpe: Sharpe du train (pour consistency), None si on calcule train
    
    Returns:
        float: Score composite (plus élevé = meilleur)
    """
    df_trades = bt_result['trades']
    df_days = bt_result['days']
    
    # Calculer métriques de base
    n_trades = len(df_trades)
    
    # Filtre: trop peu de trades = pas fiable
    if n_trades < 30:
        return -999
    
    # Sharpe ratio
    sharpe = bt_result.get('sharpe_ratio', 0)
    if pd.isna(sharpe) or np.isinf(sharpe):
        sharpe = 0
    
    # Max Drawdown
    df_days_copy = df_days.copy()
    df_days_copy['cummax'] = df_days_copy['wallet'].cummax()
    df_days_copy['drawdown_pct'] = (df_days_copy['wallet'] - df_days_copy['cummax']) / df_days_copy['cummax']
    max_dd = abs(df_days_copy['drawdown_pct'].min()) * 100
    
    # Calmar Ratio (return / max_dd)
    final_wallet = df_days['wallet'].iloc[-1]
    total_return = (final_wallet / INITIAL_WALLET - 1) * 100
    calmar = total_return / max(max_dd, 1.0)  # Éviter division par 0
    
    # Win Rate
    df_trades_copy = df_trades.copy()
    if 'trade_result' not in df_trades_copy.columns:
        df_trades_copy['trade_result'] = (
            df_trades_copy["close_trade_size"] -
            df_trades_copy["open_trade_size"] -
            df_trades_copy["open_fee"] -
            df_trades_copy["close_fee"]
        )
    win_rate = (df_trades_copy['trade_result'] > 0).mean()
    
    # Profit Factor
    gross_profit = df_trades_copy[df_trades_copy['trade_result'] > 0]['trade_result'].sum()
    gross_loss = abs(df_trades_copy[df_trades_copy['trade_result'] < 0]['trade_result'].sum())
    profit_factor = gross_profit / max(gross_loss, 1.0) if gross_loss > 0 else gross_profit
    profit_factor_normalized = min(profit_factor / 2, 1)  # Cap à 1
    
    # Consistency (train vs test)
    if train_sharpe is not None:
        consistency = 1 - abs(train_sharpe - sharpe) / max(0.1, abs(train_sharpe))
        consistency = max(0, consistency)  # Clamp à 0
    else:
        consistency = 0  # Pas de consistency pour train
    
    # Score composite
    if train_sharpe is None:  # Train
        score = (
            sharpe * 0.35 +
            calmar * 0.25 +
            (1 - min(max_dd, 100) / 100) * 0.20 +
            win_rate * 0.10 +
            profit_factor_normalized * 0.10
        )
    else:  # Test
        score = (
            sharpe * 0.30 +
            consistency * 0.25 +
            calmar * 0.20 +
            (1 - min(max_dd, 100) / 100) * 0.15 +
            win_rate * 0.05 +
            profit_factor_normalized * 0.05
        )
    
    return score

print("✅ Fonction calculate_composite_score définie")

In [ ]:
# Walk-Forward Optimization Loop
wf_results = []

print("\n🚀 Démarrage Walk-Forward Optimization...\n")
print("=" * 80)

total_iterations = len(WF_FOLDS) * len(grid_combinations) * 2  # × 2 pour fixed + adaptive
pbar = tqdm(total=total_iterations, desc="Walk-Forward Progress")

for fold in WF_FOLDS:
    fold_name = fold["name"]
    print(f"\n📂 {fold_name}")
    print(f"   Train: {fold['train_start']} → {fold['train_end']}")
    print(f"   Test:  {fold['test_start']} → {fold['test_end']}")
    
    # Filtrer données par période
    df_list_train = filter_df_list_by_dates(df_list_full, fold['train_start'], fold['train_end'])
    df_list_test = filter_df_list_by_dates(df_list_full, fold['test_start'], fold['test_end'])
    
    df_btc_train = filter_df_by_dates(df_btc_full, fold['train_start'], fold['train_end'])
    df_btc_test = filter_df_by_dates(df_btc_full, fold['test_start'], fold['test_end'])
    
    # ⚠️ IMPORTANT: Calculer régimes par fold (évite lookahead bias)
    regime_train = calculate_regime_series(df_btc_train, confirm_n=12)
    regime_test = calculate_regime_series(df_btc_test, confirm_n=12)
    
    for combo_idx, (ma_window, envelopes, size, stop_loss) in enumerate(grid_combinations):
        # Préparer params_coin
        params_coin = {}
        for pair in PAIRS:
            params_coin[pair] = {
                "src": "close",
                "ma_base_window": ma_window,
                "envelopes": envelopes,
                "size": size / BACKTEST_LEVERAGE
            }
        
        # === TRAIN ===
        # 1. Fixed
        adapter_fixed = FixedParamsAdapter(params_coin)
        bt_train_fixed = run_single_backtest(
            df_list_train, oldest_pair, params_coin, stop_loss, adapter_fixed
        )
        score_train_fixed = calculate_composite_score(bt_train_fixed)
        pbar.update(1)
        
        # 2. Adaptive
        adapter_adaptive_train = RegimeBasedAdapter(
            base_params=params_coin,
            regime_series=regime_train,
            regime_params=DEFAULT_PARAMS,
            multipliers={'envelope_std': True},
            base_std=0.10
        )
        bt_train_adaptive = run_single_backtest(
            df_list_train, oldest_pair, params_coin, stop_loss, adapter_adaptive_train
        )
        score_train_adaptive = calculate_composite_score(bt_train_adaptive)
        pbar.update(1)
        
        # === TEST ===
        # 1. Fixed
        adapter_fixed_test = FixedParamsAdapter(params_coin)
        bt_test_fixed = run_single_backtest(
            df_list_test, oldest_pair, params_coin, stop_loss, adapter_fixed_test
        )
        sharpe_train_fixed = bt_train_fixed.get('sharpe_ratio', 0)
        score_test_fixed = calculate_composite_score(bt_test_fixed, sharpe_train_fixed)
        
        # 2. Adaptive
        adapter_adaptive_test = RegimeBasedAdapter(
            base_params=params_coin,
            regime_series=regime_test,
            regime_params=DEFAULT_PARAMS,
            multipliers={'envelope_std': True},
            base_std=0.10
        )
        bt_test_adaptive = run_single_backtest(
            df_list_test, oldest_pair, params_coin, stop_loss, adapter_adaptive_test
        )
        sharpe_train_adaptive = bt_train_adaptive.get('sharpe_ratio', 0)
        score_test_adaptive = calculate_composite_score(bt_test_adaptive, sharpe_train_adaptive)
        
        # Stocker résultats
        wf_results.append({
            "fold": fold_name,
            "combo_idx": combo_idx,
            "ma_window": ma_window,
            "envelopes": str(envelopes),
            "size": size,
            "stop_loss": stop_loss,
            "adaptive": False,
            "train_wallet": bt_train_fixed['wallet'],
            "train_sharpe": sharpe_train_fixed,
            "train_score": score_train_fixed,
            "train_trades": len(bt_train_fixed['trades']),
            "test_wallet": bt_test_fixed['wallet'],
            "test_sharpe": bt_test_fixed.get('sharpe_ratio', 0),
            "test_score": score_test_fixed,
            "test_trades": len(bt_test_fixed['trades']),
        })
        
        wf_results.append({
            "fold": fold_name,
            "combo_idx": combo_idx,
            "ma_window": ma_window,
            "envelopes": str(envelopes),
            "size": size,
            "stop_loss": stop_loss,
            "adaptive": True,
            "train_wallet": bt_train_adaptive['wallet'],
            "train_sharpe": sharpe_train_adaptive,
            "train_score": score_train_adaptive,
            "train_trades": len(bt_train_adaptive['trades']),
            "test_wallet": bt_test_adaptive['wallet'],
            "test_sharpe": bt_test_adaptive.get('sharpe_ratio', 0),
            "test_score": score_test_adaptive,
            "test_trades": len(bt_test_adaptive['trades']),
        })

pbar.close()
print("\n" + "=" * 80)
print("✅ Walk-Forward Optimization terminée\n")

In [ ]:
# Créer DataFrame des résultats
df_wf_results = pd.DataFrame(wf_results)

# Calculer moyenne des scores sur les folds
df_wf_avg = df_wf_results.groupby(['combo_idx', 'ma_window', 'envelopes', 'size', 'stop_loss', 'adaptive']).agg({
    'train_score': 'mean',
    'test_score': 'mean',
    'train_sharpe': 'mean',
    'test_sharpe': 'mean',
    'train_trades': 'sum',
    'test_trades': 'sum',
}).reset_index()

# Calculer consistency
df_wf_avg['consistency'] = 1 - abs(df_wf_avg['train_sharpe'] - df_wf_avg['test_sharpe']) / df_wf_avg['train_sharpe'].abs().clip(lower=0.1)
df_wf_avg['consistency'] = df_wf_avg['consistency'].clip(lower=0)

# Score final = moyenne test_score
df_wf_avg = df_wf_avg.sort_values('test_score', ascending=False)

print("✅ Résultats agrégés")
print(f"   Total configurations testées: {len(df_wf_avg)}")
print(f"   Folds par configuration: {len(WF_FOLDS)}")

In [ ]:
# Top 10 configurations
print("\n🏆 TOP 10 CONFIGURATIONS (par Test Score moyen)\n")
print("=" * 120)

top10 = df_wf_avg.head(10)
for idx, row in top10.iterrows():
    print(f"#{idx+1}")
    print(f"   MA: {row['ma_window']}, Env: {row['envelopes']}, Size: {row['size']}, SL: {row['stop_loss']}, Adaptive: {row['adaptive']}")
    print(f"   Train Sharpe: {row['train_sharpe']:.2f}, Test Sharpe: {row['test_sharpe']:.2f}, Consistency: {row['consistency']:.2f}")
    print(f"   Train Score: {row['train_score']:.3f}, Test Score: {row['test_score']:.3f}")
    print(f"   Trades: Train={row['train_trades']}, Test={row['test_trades']}")
    print()

# Identifier le meilleur
best_config = top10.iloc[0]
print("\n" + "=" * 120)
print(f"✅ MEILLEURE CONFIGURATION:")
print(f"   MA: {best_config['ma_window']}")
print(f"   Envelopes: {best_config['envelopes']}")
print(f"   Size: {best_config['size']}")
print(f"   Stop Loss: {best_config['stop_loss']}")
print(f"   Adaptive: {best_config['adaptive']}")
print(f"   Test Score: {best_config['test_score']:.3f}")

In [ ]:
# Comparaison Phase A (8 paires) vs Phase B (28 paires)
print("📊 COMPARAISON PHASE A vs PHASE B")
print("=" * 80)

for idx, row_portfolio in df_portfolio.iterrows():
    config_idx = int(row_portfolio['config'].replace('Config#', '')) - 1
    row_phase_a = top3.iloc[config_idx]
    
    print(f"\n{row_portfolio['config']}:")
    print(f"   MA={row_portfolio['ma_window']}, Env={row_portfolio['envelopes']}, Adaptive={row_portfolio['adaptive']}")
    print(f"   Phase A (8 paires):  Score={row_phase_a['test_score']:.3f}, Sharpe={row_phase_a['test_sharpe']:.2f}")
    print(f"   Phase B (28 paires): Score={row_portfolio['score']:.3f}, Sharpe={row_portfolio['sharpe']:.2f}")
    
    # Vérifier consistency
    score_diff = abs(row_portfolio['score'] - row_phase_a['test_score'])
    sharpe_diff = abs(row_portfolio['sharpe'] - row_phase_a['test_sharpe'])
    
    if sharpe_diff <= 0.5:
        print(f"   ✅ ROBUSTE: Sharpe diff = {sharpe_diff:.2f} (≤0.5)")
    else:
        print(f"   ⚠️  DIVERGENCE: Sharpe diff = {sharpe_diff:.2f} (>0.5)")

# Sélectionner la meilleure config pour hold-out
best_config_portfolio = df_portfolio.iloc[0]

print("\n" + "=" * 80)
print(f"🏆 MEILLEURE CONFIG POUR HOLD-OUT:")
print(f"   {best_config_portfolio['config']}")
print(f"   MA: {best_config_portfolio['ma_window']}")
print(f"   Envelopes: {best_config_portfolio['envelopes']}")
print(f"   Size: {best_config_portfolio['size']}")
print(f"   Stop Loss: {best_config_portfolio['stop_loss']}")
print(f"   Adaptive: {best_config_portfolio['adaptive']}")
print(f"   Score Portfolio: {best_config_portfolio['score']:.3f}")
print("=" * 80)

In [ ]:
# Tester top-3 configs sur 28 paires
top3 = df_wf_avg.head(3)

portfolio_results = []

for idx, row in top3.iterrows():
    config_name = f"Config#{idx+1}"
    print(f"\n🔄 Test {config_name}: MA={row['ma_window']}, Env={row['envelopes']}, Adaptive={row['adaptive']}")
    
    # Préparer params
    params_coin_28 = {}
    for pair in df_list_full_28.keys():
        params_coin_28[pair] = {
            "src": "close",
            "ma_base_window": int(row['ma_window']),
            "envelopes": eval(row['envelopes']),
            "size": float(row['size']) / BACKTEST_LEVERAGE
        }
    
    # Adapter
    if row['adaptive']:
        adapter = RegimeBasedAdapter(
            base_params=params_coin_28,
            regime_series=regime_series_full_28,
            regime_params=DEFAULT_PARAMS,
            multipliers={'envelope_std': True},
            base_std=0.10
        )
    else:
        adapter = FixedParamsAdapter(params_coin_28)
    
    # Run backtest
    result = run_single_backtest(
        df_list_full_28, oldest_pair_28, params_coin_28,
        float(row['stop_loss']), adapter
    )
    
    # Métriques
    final_wallet = result['wallet']
    sharpe = result.get('sharpe_ratio', 0)
    n_trades = len(result['trades'])
    
    # Score composite (comme Phase A)
    score = calculate_composite_score(result)
    
    portfolio_results.append({
        'config': config_name,
        'ma_window': int(row['ma_window']),
        'envelopes': row['envelopes'],
        'size': float(row['size']),
        'stop_loss': float(row['stop_loss']),
        'adaptive': bool(row['adaptive']),
        'wallet': final_wallet,
        'sharpe': sharpe,
        'n_trades': n_trades,
        'score': score,
    })
    
    print(f"   Wallet: ${final_wallet:.2f}, Sharpe: {sharpe:.2f}, Trades: {n_trades}, Score: {score:.3f}")

df_portfolio = pd.DataFrame(portfolio_results).sort_values('score', ascending=False)

print("\n" + "=" * 80)
print("✅ Validation portfolio complétée\n")

In [ ]:
# Charger données pour 28 paires
df_list_full_28 = {}
print("\n📥 Chargement des 28 paires...")
for pair in tqdm(PAIRS_FULL, desc="Paires"):
    try:
         # IMPORTANT: Charger jusqu'à la fin pour avoir les données hold-out
        df = exchange.load_data(pair, "1h", start_date="2020-01-01", end_date="2024-12-31")
        df_list_full_28[pair] = df
    except FileNotFoundError:
        print(f"⚠️  {pair} non disponible, ignoré")

oldest_pair_28 = min(df_list_full_28, key=lambda p: df_list_full_28[p].index.min())

# Régimes sur BTC (même période)
df_btc_full_28 = exchange.load_data("BTC/USDT:USDT", "1h", 
                                     start_date=PERIODS['train_full']['start'], 
                                     end_date=PERIODS['train_full']['end'])
regime_series_full_28 = calculate_regime_series(df_btc_full_28, confirm_n=12)

print(f"\n✅ {len(df_list_full_28)} paires chargées (sur 28 demandées)")
print(f"   Période: 2020-01-01 → 2024-12-31 (couvre tous les cycles)")

In [ ]:
# Sauvegarder les résultats
output_path = "backtest_comparison_results.csv"
comparator.save_comparison(output_path)
print(f"\n💾 Résultats complets sauvegardés: {output_path}")

✅ Comparaison sauvegardée: backtest_comparison_results.csv

💾 Résultats complets sauvegardés: backtest_comparison_results.csv


In [34]:
# Paires complètes du live bot (28 paires)PAIRS_FULL = [    "BTC/USDT:USDT", "ETH/USDT:USDT", "BNB/USDT:USDT", "SOL/USDT:USDT",    "XRP/USDT:USDT", "DOGE/USDT:USDT", "ADA/USDT:USDT", "AVAX/USDT:USDT",    "SHIB/USDT:USDT", "DOT/USDT:USDT", "LINK/USDT:USDT", "MATIC/USDT:USDT",    "UNI/USDT:USDT", "ATOM/USDT:USDT", "LTC/USDT:USDT", "ETC/USDT:USDT",    "APT/USDT:USDT", "ARB/USDT:USDT", "OP/USDT:USDT", "NEAR/USDT:USDT",    "FIL/USDT:USDT", "INJ/USDT:USDT", "IMX/USDT:USDT", "RUNE/USDT:USDT",    "SUSHI/USDT:USDT", "TRX/USDT:USDT", "AAVE/USDT:USDT", "CRV/USDT:USDT",]print(f"📊 PHASE B - VALIDATION PORTFOLIO COMPLET")print("=" * 80)print(f"   Paires: {len(PAIRS_FULL)} (portfolio complet)")print(f"   Période: {PERIODS['train_full']['start']} → {PERIODS['train_full']['end']} (BULL 2020-21, BEAR 2022, RECOVERY 2023, BULL 2024)")print(f"   Top configs à tester: 3")print(f"\n⚠️  Cette phase valide que les configs tiennent sur le portfolio complet")print("=" * 80)

📊 PHASE B - VALIDATION PORTFOLIO COMPLET
   Paires: 28 (portfolio complet)
   Période: 2022-01-01 → 2024-06-30
   Top configs à tester: 3

⚠️  Cette phase valide que les configs tiennent sur le portfolio complet


## 3.5️⃣ Phase B - Validation Portfolio complet (28 paires)

Teste le **top-3** des configs sur les **28 paires complètes** (même période train) pour vérifier que les résultats tiennent sur le portfolio complet.

## 4️⃣ Validation Hold-out finale

Test **UNE SEULE FOIS** sur les données de hold-out (2024 H2) pour vérifier qu'il n'y a pas d'overfitting.

In [35]:
print("\n⚠️  VALIDATION HOLD-OUT FINALE (28 paires)")
print("=" * 80)
print(f"Période: {PERIODS['holdout']['start']} → {PERIODS['holdout']['end']}")
print(f"⚠️  Cette validation ne peut être exécutée qu'UNE SEULE FOIS !\n")

# Filtrer données hold-out (28 paires)
df_list_holdout_28 = filter_df_list_by_dates(df_list_full_28, PERIODS['holdout']['start'], PERIODS['holdout']['end'])
df_btc_holdout = exchange.load_data("BTC/USDT:USDT", "1h", 
                                     start_date=PERIODS['holdout']['start'], 
                                     end_date=PERIODS['holdout']['end'])
regime_holdout = calculate_regime_series(df_btc_holdout, confirm_n=12)

# Utiliser la meilleure config de Phase B (portfolio complet)
best_cfg = df_portfolio.iloc[0]  # Meilleure config selon score Phase B

# Préparer params pour 28 paires
params_coin_holdout = {}
for pair in df_list_holdout_28.keys():
    params_coin_holdout[pair] = {
        "src": "close",
        "ma_base_window": int(best_cfg['ma_window']),
        "envelopes": eval(best_cfg['envelopes']) if isinstance(best_cfg['envelopes'], str) else best_cfg['envelopes'],
        "size": float(best_cfg['size']) / BACKTEST_LEVERAGE
    }

# Test hold-out
if best_cfg['adaptive']:
    adapter_holdout = RegimeBasedAdapter(
        base_params=params_coin_holdout,
        regime_series=regime_holdout,
        regime_params=DEFAULT_PARAMS,
        multipliers={'envelope_std': True},
        base_std=0.10
    )
else:
    adapter_holdout = FixedParamsAdapter(params_coin_holdout)

bt_holdout = run_single_backtest(
    df_list_holdout_28, oldest_pair_28, params_coin_holdout,
    float(best_cfg['stop_loss']), adapter_holdout
)

# Calculer métriques hold-out
holdout_sharpe = bt_holdout.get('sharpe_ratio', 0)
holdout_wallet = bt_holdout['wallet']
holdout_perf = (holdout_wallet / INITIAL_WALLET - 1) * 100
holdout_trades = len(bt_holdout['trades'])

# Comparaison avec Phase B (portfolio)
portfolio_sharpe = best_cfg['sharpe']

print(f"\n📊 RÉSULTATS HOLD-OUT:")
print(f"   Wallet final: ${holdout_wallet:.2f}")
print(f"   Performance: {holdout_perf:+.2f}%")
print(f"   Sharpe Ratio: {holdout_sharpe:.2f}")
print(f"   Nombre de trades: {holdout_trades}")

print(f"\n📊 COMPARAISON:")
print(f"   Phase B Portfolio Sharpe: {portfolio_sharpe:.2f}")
print(f"   Hold-out Sharpe:          {holdout_sharpe:.2f}")

# Validation
sharpe_diff = abs(holdout_sharpe - portfolio_sharpe)

if sharpe_diff <= 0.5:
    print(f"\n✅ VALIDATION RÉUSSIE: Hold-out Sharpe ≈ Portfolio Sharpe (diff={sharpe_diff:.2f})")
    print(f"   Pas d'overfitting détecté. Configuration robuste sur 28 paires.")
elif sharpe_diff <= 1.0:
    print(f"\n⚠️  WARNING: Hold-out Sharpe diverge légèrement (diff={sharpe_diff:.2f})")
    print(f"   Overfitting possible mais acceptable.")
else:
    print(f"\n❌ ÉCHEC: Hold-out Sharpe diverge fortement (diff={sharpe_diff:.2f})")
    print(f"   Overfitting détecté ! Réviser les paramètres.")

print("\n" + "=" * 80)


⚠️  VALIDATION HOLD-OUT FINALE (28 paires)
Période: 2024-07-01 → 2024-12-31
⚠️  Cette validation ne peut être exécutée qu'UNE SEULE FOIS !



d:\Python\Cryptobots\Backtest-Tools-V2\strategies\envelopes\../..\utilities\strategies\envelopeMulti_v2.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(
d:\Python\Cryptobots\Backtest-Tools-V2\strategies\envelopes\../..\utilities\strategies\envelopeMulti_v2.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(
d:\Python\Cryptobots\Backtest-Tools-V2\strategies\envelopes\../..\utilities\strategies\envelopeMulti_v2.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

KeyError: 'day'

In [36]:
# Vérification des données hold-out avant le test
print("🔍 Vérification des données hold-out...")
print(f"Nombre de paires chargées: {len(df_list_holdout_28)}")
for pair, df in list(df_list_holdout_28.items())[:3]:
    print(f"   {pair}: {len(df)} barres ({df.index.min()} → {df.index.max()})")
print(f"\nBTC régime hold-out: {len(regime_holdout)} barres")
print(f"Régimes: {regime_holdout.value_counts().to_dict()}")

🔍 Vérification des données hold-out...
Nombre de paires chargées: 27
   BTC/USDT:USDT: 0 barres (NaT → NaT)
   ETH/USDT:USDT: 0 barres (NaT → NaT)
   BNB/USDT:USDT: 0 barres (NaT → NaT)

BTC régime hold-out: 4415 barres
Régimes: {<Regime.BULL: 'bull'>: 2348, <Regime.BEAR: 'bear'>: 1299, <Regime.RECOVERY: 'recovery'>: 768}


## 5️⃣ Résultats et visualisation

In [ ]:
# Scatter plot : Train Sharpe vs Test Sharpe
fig, ax = plt.subplots(figsize=(10, 8))

# Séparer Fixed et Adaptive
df_fixed = df_wf_avg[df_wf_avg['adaptive'] == False]
df_adaptive = df_wf_avg[df_wf_avg['adaptive'] == True]

ax.scatter(df_fixed['train_sharpe'], df_fixed['test_sharpe'], 
           alpha=0.6, s=100, label='Fixed', marker='o')
ax.scatter(df_adaptive['train_sharpe'], df_adaptive['test_sharpe'], 
           alpha=0.6, s=100, label='Adaptive', marker='^')

# Ligne y=x (pas d'overfitting)
max_sharpe = max(df_wf_avg['train_sharpe'].max(), df_wf_avg['test_sharpe'].max())
min_sharpe = min(df_wf_avg['train_sharpe'].min(), df_wf_avg['test_sharpe'].min())
ax.plot([min_sharpe, max_sharpe], [min_sharpe, max_sharpe], 
        'r--', alpha=0.5, label='No overfitting (y=x)')

# Zone acceptable (±0.5)
ax.fill_between([min_sharpe, max_sharpe], 
                 [min_sharpe - 0.5, max_sharpe - 0.5],
                 [min_sharpe + 0.5, max_sharpe + 0.5],
                 alpha=0.1, color='green', label='Acceptable zone (±0.5)')

ax.set_xlabel('Train Sharpe Ratio', fontsize=12)
ax.set_ylabel('Test Sharpe Ratio', fontsize=12)
ax.set_title('Train vs Test Sharpe Ratio (Overfitting Detection)', fontsize=14, fontweight='bold')
ax.legend(loc='upper left')
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(f'train_vs_test_sharpe_{timestamp}.png', dpi=150)
plt.show()

print(f"💾 Graphique sauvegardé: train_vs_test_sharpe_{timestamp}.png")

In [ ]:
# Bar chart : Top 10 configurations par test score
fig, ax = plt.subplots(figsize=(14, 8))

top10_display = df_wf_avg.head(10).copy()
top10_display['config_label'] = (
    'MA=' + top10_display['ma_window'].astype(str) + 
    ', Size=' + top10_display['size'].astype(str) +
    ', ' + top10_display['adaptive'].map({True: 'Adapt', False: 'Fixed'})
)

x = range(len(top10_display))
width = 0.35

bars1 = ax.bar([i - width/2 for i in x], top10_display['train_score'], 
               width, label='Train Score', alpha=0.8, color='steelblue')
bars2 = ax.bar([i + width/2 for i in x], top10_display['test_score'], 
               width, label='Test Score', alpha=0.8, color='coral')

ax.set_xlabel('Configuration', fontsize=12)
ax.set_ylabel('Composite Score', fontsize=12)
ax.set_title('Top 10 Configurations by Test Score', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(top10_display['config_label'], rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(f'top10_scores_{timestamp}.png', dpi=150)
plt.show()

print(f"💾 Graphique sauvegardé: top10_scores_{timestamp}.png")

In [ ]:
# Sauvegarder tous les résultats
df_wf_results.to_csv(f"wf_results_detailed_{timestamp}.csv", index=False)
df_wf_avg.to_csv(f"wf_results_summary_{timestamp}.csv", index=False)

# Sauvegarder meilleure config en JSON
import json

best_config_export = {
    "ma_base_window": int(best_config['ma_window']),
    "envelopes": eval(best_config['envelopes']),
    "size": float(best_config['size']),
    "stop_loss": float(best_config['stop_loss']),
    "adaptive": bool(best_config['adaptive']),
    "train_sharpe": float(best_config['train_sharpe']),
    "test_sharpe": float(best_config['test_sharpe']),
    "test_score": float(best_config['test_score']),
    "holdout_sharpe": float(holdout_sharpe),
    "holdout_perf": float(holdout_perf),
    "timestamp": timestamp,
}

with open(f"best_config_{timestamp}.json", 'w') as f:
    json.dump(best_config_export, f, indent=2)

print(f"\n💾 Résultats sauvegardés:")
print(f"   - wf_results_detailed_{timestamp}.csv (tous les backtests)")
print(f"   - wf_results_summary_{timestamp}.csv (moyennes par config)")
print(f"   - best_config_{timestamp}.json (meilleure configuration)")

## 🎯 Recommandation finale

In [ ]:
print("\n" + "=" * 80)
print("🎯 RECOMMANDATION FINALE")
print("=" * 80)

# Utiliser df_portfolio directement
best_cfg_final = df_portfolio.iloc[0]

print(f"\n✅ Meilleure configuration identifiée (validée sur 28 paires):")
print(f"   Config: {best_cfg_final['config']}")
print(f"   ma_base_window: {best_cfg_final['ma_window']}")
print(f"   envelopes: {best_cfg_final['envelopes']}")
print(f"   size: {best_cfg_final['size']}")
print(f"   stop_loss: {best_cfg_final['stop_loss']}")
print(f"   adaptive: {best_cfg_final['adaptive']}")

print(f"\n📊 Performance validée:")
print(f"   Phase A (8 paires) - Test Score: {top3.iloc[0]['test_score']:.3f}, Sharpe: {top3.iloc[0]['test_sharpe']:.2f}")
print(f"   Phase B (28 paires) - Score: {best_cfg_final['score']:.3f}, Sharpe: {best_cfg_final['sharpe']:.2f}")
print(f"   Hold-out (28 paires) - Sharpe: {holdout_sharpe:.2f}, Perf: {holdout_perf:+.2f}%")

if sharpe_diff <= 0.5:
    print(f"\n✅ Validation: Configuration robuste (pas d'overfitting)")
    print(f"   ✓ Testée sur 8 paires (Walk-Forward)")
    print(f"   ✓ Validée sur 28 paires (Phase B)")
    print(f"   ✓ Hold-out confirmé (2024-H2)")
    print(f"   → RECOMMANDÉ pour mise en production")
elif sharpe_diff <= 1.0:
    print(f"\n⚠️  Validation: Overfitting léger détecté")
    print(f"   → Utiliser avec prudence, surveiller en live")
else:
    print(f"\n❌ Validation: Overfitting significatif")
    print(f"   → NE PAS utiliser en production")
    print(f"   → Réduire la complexité du grid ou augmenter les données")

print(f"\n📝 Prochaines étapes:")
print(f"   1. Appliquer la config dans multi_envelope.ipynb (28 paires)")
print(f"   2. Valider sur paper trading / forward test")
print(f"   3. Si résultats conformes → Déployer en production")

print(f"\n💡 Note:")
print(f"   Cette config a été optimisée sur un échantillon stratifié (8 paires)")
print(f"   puis validée sur le portfolio complet (28 paires) + hold-out.")
print(f"   Méthodologie robuste anti-overfitting.")
print(f"\n⚠️  IMPORTANT: Paramètres GLOBAUX (identiques pour toutes les cryptos)")
print(f"   Pour optimiser par profil (majors/mid-caps/volatiles/low), voir Option 1")

print("\n" + "=" * 80)